In [1]:
pip install requests beautifulsoup4 pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

driver.get('https://www.amazon.com')

# Function to search Amazon and extract product URL and ASIN
def search_amazon(product_title, driver):
    # Construct the search URL
    search_url = f"https://www.amazon.com/s?k={'+'.join(product_title.split())}"

    # Load the search page
    driver.get(search_url)

    # Wait for the page to load
    time.sleep(3)

    try:
        # Find the first product in the search results
        first_product = driver.find_element(By.CSS_SELECTOR, 'div.s-main-slot div[data-component-type="s-search-result"]')
        link = first_product.find_element(By.CSS_SELECTOR, 'a.a-link-normal')

        # Get the product URL
        product_url = link.get_attribute('href')
        
        # Extract ASIN from the URL
        if "/dp/" in product_url:
            asin = product_url.split('/dp/')[1][:10]  # Extract ASIN from the URL
        else:
            asin = None
        
        return product_url, asin
    except Exception as e:
        print(f"Error fetching data for {product_title}: {e}")
        return None, None

# Function to load product titles from Excel file
def load_product_titles(excel_file_path):
    df = pd.read_excel(excel_file_path)
    return df['Title'].tolist()

# Main function to scrape data and save to an Excel file
def main():
    # Specify the path to your Excel file
    excel_file_path = "/Users/user/Downloads/Amazon Scrapping Interview Task (1) (3).xlsx"  # Update this path

    # Load product titles from the Excel file
    product_titles = load_product_titles(excel_file_path)

    # Data storage for results
    results = []

    # Iterate through each product title and search
    for title in product_titles:
        if isinstance(title, str) and title.strip():  # Check for valid title
            print(f"Searching for: {title}")
            amazon_link, asin = search_amazon(title, driver)
            results.append({
                "Product Title": title,
                "Amazon Link": amazon_link,
                "ASIN": asin
            })
            time.sleep(2)  # Pause between requests to avoid being blocked
        else:
            print(f"Skipping invalid title: {title}")

    # Close the WebDriver session
    driver.quit()

    # Convert results to a DataFrame and save to Excel
    df_results = pd.DataFrame(results)
    output_file_path = "amazon_product_data.xlsx"
    df_results.to_excel(output_file_path, index=False)
    print(f"Data saved to {output_file_path}")

if __name__ == "__main__":
    main()


Searching for: Adeptus Mechanicus Sicarians Warhammer 40,000
Searching for: Games Workshop 99070101036" Dark Angels Primaris Upgrades Miniature
Searching for: Games Workshop - Warhammer 40,000 - Kill Team: Hand Of The Archon [video game]
Searching for: Games Workshop - Warhammer 40,000 - Leagues of Votann: Hearthkyn Warriors
Searching for: Games Workshop - 99120112043 - Warhammer 40,000 - Combat Patrol: Drukhari
Error fetching data for Games Workshop - 99120112043 - Warhammer 40,000 - Combat Patrol: Drukhari: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.s-main-slot div[data-component-type="s-search-result"]"}
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100fac248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001